This lab on Cross-Validation is a python adaptation of p. 190-194 of "Introduction to Statistical Learning
with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. From
<t>http://www.science.smith.edu/~jcrouser/SDS293/labs/</t>

### 5.3.1 The Validation Set Approach

The aim is to use the validation set approach in order to estimate the test error rates that result from fitting various linear models.

In [1]:
import numpy as np                     #library for scientific computing
import matplotlib.pyplot as plt        #library for plotting data
import pandas as pd                    #library for data analysis
import sklearn.linear_model as skl_lm  #library for machine (statistical) learning
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

Consider the data set <b>Auto</b>

In [2]:
thisdata = pd.read_csv('Auto.csv', na_values='?').dropna() #read csv, DROPping Non Available data
thisdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 396
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 30.6+ KB


<b>Generate the training set:</b>
Split the set of observations into two halves, randomly choosen (use function <tt>sample()</tt>).

In [3]:
train_set = thisdata.sample(196, random_state = 1)  #random_state = set a seed

<b>Generate the test set:</b>
Take the rest of the 392 data points (use function <tt>isin</tt>).

In [4]:
test_set = thisdata[~thisdata.isin(train_set)].dropna(how = 'all')

Consider the variables <tt>horsepower</tt> and <tt>mpg</tt> in each set (use function <tt>values</tt> to plot only the values of the variables)

In [8]:
#Xtrain = train_set['horsepower']
#Ytrain = train_set['mpg']
#Xtest = test_set['horsepower']
#Ytest = test_set['mpg']
Xtrain = train_set['horsepower'].values
Ytrain = train_set['mpg'].values
Xtest = test_set['horsepower'].values
Ytest = test_set['mpg'].values
#print(Xtrain,'\n',Ytrain,'\n',Xtest,'\n',Ytest)

Give a new shape to the arrays <tt>horsepower</tt> without changing the values. Note: The "-1" in <tt>reshape(-1,1)</tt> means that we do not known the dimension and we want <tt>numpy</tt> to figure it out.

In [9]:
Xtrain = train_set['horsepower'].values.reshape(-1,1) 
Ytrain = train_set['mpg'].values
Xtest = test_set['horsepower'].values.reshape(-1,1)
Ytest = test_set['mpg'].values
#print(Xtrain,'\n',Ytrain,'\n',Xtest,'\n',Ytest)

<b>Linear regression</b>: 
We now fit a linear regression to the data to predict <tt>mpg</tt> (miles per gallon) from <tt>horsepower</tt> using only the observations corresponding to the training set. Note: we use here function <tt> LinearRegression()</tt>.

In [11]:
lm = skl_lm.LinearRegression()
model = lm.fit(Xtrain, Ytrain)
#Ytrain = (model.coef_) * Xtrain + (model.intersept_)
print(model.coef_,model.intercept_)

[-0.15964606] 40.3338030434159


We now use the <tt>predict()</tt> function to estimate the response for the test
observations, and evaluate it prediction with function <tt>mean_squared_error</tt> in <tt>sklearn.metrics</tt>.

In [13]:
predic = model.predict(Xtest) #Make prediction
#print(predic)
#Assess how good the model is
MSE = mean_squared_error(Ytest, predic)  #Error from "true" value (Ytest)
print(MSE)

23.361902892587224


<b>Quadratic regression</b>:
We use function <tt>PolynomialFeatures</tt> to estimate the test error for the polynomial and cubic regressions.

In [15]:
# Quadratic polynomial (degree=2)
poly = PolynomialFeatures(degree=2)
Xtrain2 = poly.fit_transform(Xtrain) #IMPORTANT: Puts each X-value -> 1 / X / X^2
Xtest2 = poly.fit_transform(Xtest)
print(Xtrain[0:5],'\n',Xtrain2[0:5])
print(Xtest[0:5],'\n',Xtest2[0:5])

[[ 97.]
 [ 75.]
 [ 75.]
 [112.]
 [ 67.]] 
 [[1.0000e+00 9.7000e+01 9.4090e+03]
 [1.0000e+00 7.5000e+01 5.6250e+03]
 [1.0000e+00 7.5000e+01 5.6250e+03]
 [1.0000e+00 1.1200e+02 1.2544e+04]
 [1.0000e+00 6.7000e+01 4.4890e+03]]
[[165.]
 [150.]
 [150.]
 [215.]
 [170.]] 
 [[1.0000e+00 1.6500e+02 2.7225e+04]
 [1.0000e+00 1.5000e+02 2.2500e+04]
 [1.0000e+00 1.5000e+02 2.2500e+04]
 [1.0000e+00 2.1500e+02 4.6225e+04]
 [1.0000e+00 1.7000e+02 2.8900e+04]]


In [17]:
# Make model
quadmodel = lm.fit(Xtrain2, Ytrain)
print(quadmodel.coef_,quadmodel.intercept_)
quadMSE = mean_squared_error(Ytest, quadmodel.predict(Xtest2))
print(quadMSE)

[ 0.         -0.51860376  0.00141527] 60.30223777216642
20.252690858345748


Seems that quadratic is better.... :-)

<b>Cubic regression</b>

In [18]:
# Cubic polynomial
poly = PolynomialFeatures(degree=3)

# Puts each X-value -> 1 / X / X^2 / X^3
Xtrain3 = poly.fit_transform(Xtrain)
Xtest3 = poly.fit_transform(Xtest)

cubicmodel = lm.fit(Xtrain3, Ytrain)
print(cubicmodel.coef_,cubicmodel.intercept_)
cubicMSE = mean_squared_error(Ytest, model.predict(Xtest3))
print(cubicMSE)

[ 0.00000000e+00 -6.86758109e-01  2.80449742e-03 -3.52379474e-06] 66.51996119784485
20.325609365972525


hmm... now got bit worse... shall we continue?

In [20]:
print("Polynomial order 4")
poly = PolynomialFeatures(degree=4)
Xtrain4 = poly.fit_transform(Xtrain)
Xtest4 = poly.fit_transform(Xtest)
model = lm.fit(Xtrain4, Ytrain)
MSE = mean_squared_error(Ytest, model.predict(Xtest4))
print("Coefficients for polynomial order 4")
print(model.coef_,model.intercept_)
print("and MSE:")
print(MSE)
print("Polynomial order 5")
poly = PolynomialFeatures(degree=5)
Xtrain5 = poly.fit_transform(Xtrain)
Xtest5 = poly.fit_transform(Xtest)
model = lm.fit(Xtrain5, Ytrain)
MSE = mean_squared_error(Ytest, model.predict(Xtest5))
print("Coefficients for polynomial order 5")
print(model.coef_,model.intercept_)
print("and MSE:")
print(MSE)
print("Polynomial order 6")
poly = PolynomialFeatures(degree=6)
Xtrain6 = poly.fit_transform(Xtrain)
Xtest6 = poly.fit_transform(Xtest)
model = lm.fit(Xtrain6, Ytrain)
MSE = mean_squared_error(Ytest, model.predict(Xtest6))
print("Coefficients for polynomial order 6")
print(model.coef_,model.intercept_)
print("and MSE:")
print(MSE)

Polynomial order 4
Coefficients for polynomial order 4
[ 0.00000000e+00 -4.07089951e-01 -5.26969867e-04  1.45699704e-05
 -3.61932229e-08] 57.089007717397536
and MSE:
19.82328003535277
Polynomial order 5
Coefficients for polynomial order 5
[ 0.00000000e+00  2.57078850e+00 -5.27277434e-02  4.47228844e-04
 -1.73823447e-06  2.55305510e-09] -7.010477966978165
and MSE:
19.115946589914845
Polynomial order 6
Coefficients for polynomial order 6
[ 0.00000000e+00  5.98932799e+00 -1.29674595e-01  1.32557738e-03
 -7.11271127e-06  1.93230131e-08 -2.09263717e-11] -67.12990022730582
and MSE:
18.891373340709364


<b>Note</b>: These different error rates are due to the choice of the training set. If we choose a different
training set instead, then we will obtain somewhat different errors on the
validation set. We can test this out by setting a different random seed (different <tt>random_state</tt>)

In [21]:
# Make two other subsets (training and testing)
other_train_set = thisdata.sample(196, random_state = 2)
other_test_set = thisdata[~thisdata.isin(other_train_set)].dropna(how = 'all')  

# "Extract" the variables 'horsepower' and 'mpg'
Xtrain = other_train_set['horsepower'].values.reshape(-1,1) 
Ytrain = other_train_set['mpg'].values
Xtest = other_test_set['horsepower'].values.reshape(-1,1)
Ytest = other_test_set['mpg'].values

print("Linear fit")
model = lm.fit(Xtrain, Ytrain)
print(model.coef_,model.intercept_)
print(mean_squared_error(Ytest, model.predict(Xtest)))

print("Quadratic fit")
poly = PolynomialFeatures(degree=2)
Xtrain2 = poly.fit_transform(Xtrain)
Xtest2 = poly.fit_transform(Xtest)
model = lm.fit(Xtrain2, Ytrain)
print(model.coef_,model.intercept_)
print(mean_squared_error(Ytest, model.predict(Xtest2)))

print("Cubic fit")
poly = PolynomialFeatures(degree=3)
Xtrain3 = poly.fit_transform(Xtrain)
Xtest3 = poly.fit_transform(Xtest)
model = lm.fit(Xtrain3, Ytrain)
print(model.coef_,model.intercept_)
print(mean_squared_error(Ytest, model.predict(Xtest3)))

print("Polynomial order 4")
poly = PolynomialFeatures(degree=4)
Xtrain4 = poly.fit_transform(Xtrain)
Xtest4 = poly.fit_transform(Xtest)
model = lm.fit(Xtrain4, Ytrain)
MSE = mean_squared_error(Ytest, model.predict(Xtest4))
print("Coefficients for polynomial order 4")
print(model.coef_,model.intercept_)
print("and MSE:")
print(MSE)

print("Polynomial order 5")
poly = PolynomialFeatures(degree=5)
Xtrain5 = poly.fit_transform(Xtrain)
Xtest5 = poly.fit_transform(Xtest)
model = lm.fit(Xtrain5, Ytrain)
MSE = mean_squared_error(Ytest, model.predict(Xtest5))
print("Coefficients for polynomial order 5")
print(model.coef_,model.intercept_)
print("and MSE:")
print(MSE)

print("Polynomial order 6")
poly = PolynomialFeatures(degree=6)
Xtrain6 = poly.fit_transform(Xtrain)
Xtest6 = poly.fit_transform(Xtest)
model = lm.fit(Xtrain6, Ytrain)
MSE = mean_squared_error(Ytest, model.predict(Xtest6))
print("Coefficients for polynomial order 6")
print(model.coef_,model.intercept_)
print("and MSE:")
print(MSE)

Linear fit
[-0.15097529] 39.0131086591854
25.10853905288967
Quadratic fit
[ 0.         -0.45662488  0.00120242] 56.04672548439809
19.722533470492426
Cubic fit
[ 0.00000000e+00 -6.87826381e-01  3.09360410e-03 -4.74583024e-06] 64.68435751138958
19.92136786007267
Polynomial order 4
Coefficients for polynomial order 4
[ 0.00000000e+00 -4.07089951e-01 -5.26969867e-04  1.45699704e-05
 -3.61932229e-08] 57.089007717397536
and MSE:
19.82328003535277
Polynomial order 5
Coefficients for polynomial order 5
[ 0.00000000e+00  2.57078850e+00 -5.27277434e-02  4.47228844e-04
 -1.73823447e-06  2.55305510e-09] -7.010477966978165
and MSE:
19.115946589914845
Polynomial order 6
Coefficients for polynomial order 6
[ 0.00000000e+00  5.98932799e+00 -1.29674595e-01  1.32557738e-03
 -7.11271127e-06  1.93230131e-08 -2.09263717e-11] -67.12990022730582
and MSE:
18.891373340709364


You could use here a <tt>for</tt>-loop:

In [23]:
for order in range(1,7):
    print("Polynomial order "+str(order))
    poly = PolynomialFeatures(degree=order)
    thisXtrain = poly.fit_transform(Xtrain)
    thisXtest = poly.fit_transform(Xtest)
    thismodel = lm.fit(thisXtrain, Ytrain)
    thisMSE = mean_squared_error(Ytest, thismodel.predict(thisXtest))
    print("Coefficients for polynomial order "+str(order))
    print(thismodel.coef_,thismodel.intercept_)
    print("and MSE:")
    print(thisMSE)

Polynomial order 1
Coefficients for polynomial order 1
[ 0.         -0.15097529] 39.0131086591854
and MSE:
25.10853905288967
Polynomial order 2
Coefficients for polynomial order 2
[ 0.         -0.45662488  0.00120242] 56.04672548439809
and MSE:
19.722533470492426
Polynomial order 3
Coefficients for polynomial order 3
[ 0.00000000e+00 -6.87826381e-01  3.09360410e-03 -4.74583024e-06] 64.68435751138958
and MSE:
19.92136786007267
Polynomial order 4
Coefficients for polynomial order 4
[ 0.00000000e+00 -4.07089951e-01 -5.26969867e-04  1.45699704e-05
 -3.61932229e-08] 57.089007717397536
and MSE:
19.82328003535277
Polynomial order 5
Coefficients for polynomial order 5
[ 0.00000000e+00  2.57078850e+00 -5.27277434e-02  4.47228844e-04
 -1.73823447e-06  2.55305510e-09] -7.010477966978165
and MSE:
19.115946589914845
Polynomial order 6
Coefficients for polynomial order 6
[ 0.00000000e+00  5.98932799e+00 -1.29674595e-01  1.32557738e-03
 -7.11271127e-06  1.93230131e-08 -2.09263717e-11] -67.12990022730

Using this split of the observations into a training set and a validation
set, we find that the validation set error rates for the models with linear,
quadratic, and cubic terms are 25.11, 19.72, and 19.92, respectively.

These results are consistent with our previous findings: a model that
predicts ${\tt mpg}$ using a quadratic function of ${\tt horsepower}$ performs better than
a model that involves only a linear function of ${\tt horsepower}$, and there is
little evidence in favor of a model that uses a cubic function of ${\tt horsepower}$.

### 5.3.2 Leave-One-Out Cross-Validation

The LOOCV estimate can be automatically computed for any generalized linear model using the `LeaveOneOut()` and `KFold()` functions. Note: <tt>LeaveOneOut()</tt> is equivalent to <tt>KFold(n_splits=n)</tt> and <tt>LeavePOut(p=1)</tt> where <tt>n</tt> is the number of samples.

In [24]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score  #needed for computing the errors

Lets go back to the linear model:

In [25]:
model = lm.fit(Xtrain, Ytrain)

Now, get the number of splitting iterations in the cross-validator (one-by-one leads to number of data points)

In [27]:
loo = LeaveOneOut()
X = thisdata['horsepower'].values.reshape(-1,1)
y = thisdata['mpg'].values.reshape(-1,1)
nsplits = loo.get_n_splits(X)
print(nsplits)

392


Note: for the next window: The parameter <tt>scoring</tt> designates scorer object with the scoring parameter. All scorer objects follow the convention that higher return values are better than lower return values. Thus metrics which measure the distance between the model and the data, like metrics.mean_squared_error, are available as <i>negative values</i>, thus <tt>neg_mean_squared_error</tt> which return the symmetric value of the metric.

In [28]:
crossvalidation = KFold(n_splits=392, random_state=None, shuffle=False)
scores = cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=crossvalidation, n_jobs=1)
#print(scores)
print("Folds: " + str(len(scores)) + ", MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Folds: 392, MSE: 24.231513517929226, STD: 36.79731503640535


Now the same for all the other polynomial fits:

In [29]:
for i in range(1,7):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lm.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=crossvalidation,
 n_jobs=1)
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))


Degree-1 polynomial MSE: 24.231513517929226, STD: 36.797315036405344
Degree-2 polynomial MSE: 19.248213124489745, STD: 34.998446151782346
Degree-3 polynomial MSE: 19.33498406411498, STD: 35.76513567797254
Degree-4 polynomial MSE: 19.424430307079398, STD: 35.68335275228356
Degree-5 polynomial MSE: 19.033198669299846, STD: 35.31730233206771
Degree-6 polynomial MSE: 18.998016005583708, STD: 35.3208471634935


Here we see a sharp drop in the estimated test MSE between
the linear and quadratic fits, but then no clear improvement from using
higher-order polynomials. What should we conclude? (Remember the balance between accuracy and complexity... :-))

### 5.3.3 k-Fold Cross-Validation

Instead of using <tt>n</tt> as the number of "folds" in the <tt>KFold</tt> function we can use (preferably!) a small number, say k=10, on the `Auto` data set. 

In [30]:
thiscrossvalidation = KFold(n_splits=10, shuffle=False)

Now, we do polynomial regression for that new kind of cross-validation (<tt>cv=thiscrossvalidation</tt>). Note: notice how faster <tt>Kfold</tt> is in comparison with <tt>LOOCV</tt>

In [31]:
for i in range(1,7):
    poly = PolynomialFeatures(degree=i)
    X_current = poly.fit_transform(X)
    model = lm.fit(X_current, y)
    scores = cross_val_score(model, X_current, y, scoring="neg_mean_squared_error", cv=thiscrossvalidation,
 n_jobs=1)
    print("Degree-"+str(i)+" polynomial MSE: " + str(np.mean(np.abs(scores))) + ", STD: " + str(np.std(scores)))

Degree-1 polynomial MSE: 27.439933652339864, STD: 14.510250711281133
Degree-2 polynomial MSE: 21.235840055801567, STD: 11.797327528897465
Degree-3 polynomial MSE: 21.336606183382038, STD: 11.844339714535932
Degree-4 polynomial MSE: 21.353886969306874, STD: 11.986332290218549
Degree-5 polynomial MSE: 20.905558736691837, STD: 12.185388793921343
Degree-6 polynomial MSE: 20.780544653507675, STD: 12.135161876520757


We still see little evidence that using cubic or higher-order polynomial terms leads to lower test error than simply using a quadratic fit.

<b>Can you apply CV to your data set? Try it!</b>

In [104]:
# Your code here